# Path of Exile game stats data eda

dates: 2022.04.30 ~ 

This file contains an analysis processing of the game 'Path of Exile' data.

###### Origins of datas from here: 
poe_stats : https://www.kaggle.com/datasets/gagazet/path-of-exile-league-statistic

Data contains stats of 59000 players, from 4th August of 2017 and before now.

---

This file made for EDA after altering the original data. (about class and rank)

본격적인 EDA로 첫 EDA 주제는 ladder(game mode)에 따라 분석을 진행하였으며,

이번에는 다음 분석으로 클래스별로 나누어 볼 예정입니다.


# Dataset and libraries 

## data and libraries import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

df = pd.read_csv('poe_stats_2revisedrank.csv')

In [3]:
df.head()

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
1,1,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
2,1,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
3,1,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1


In [4]:
df.shape

(59776, 17)

## data describes

이루어진 컬럼들, 그리고 간단한 통계량 결과 내어놓기

In [5]:
# columns name

df.columns

Index(['rank', 'dead', 'online', 'name', 'level', 'class', 'id', 'experience',
       'account', 'challenges', 'twitch', 'ladder', 'special_class',
       'stat_Str', 'stat_Int', 'stat_Dex', 'stats_num'],
      dtype='object')

In [6]:
df.describe()

,rank,level,experience,challenges,special_class,stat_Str,stat_Int,stat_Dex,stats_num
count,59776.000000,59776.000000,5.977600e+04,59776.000000,59776.000000,59776.000000,59776.000000,59776.000000,59776.000000
mean,7472.536386,85.422578,1.825786e+09,25.759569,0.991702,0.582207,0.418027,0.513450,1.513684
std,4314.060306,11.288017,1.094748e+09,8.440888,0.090713,0.493200,0.493239,0.499823,0.614134
min,1.000000,53.000000,7.790982e+07,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3736.750000,78.000000,7.774697e+08,19.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,7472.500000,90.000000,1.934010e+09,25.000000,1.000000,1.000000,0.000000,1.000000,1.000000
75%,11208.250000,95.000000,2.876117e+09,33.000000,1.000000,1.000000,1.000000,1.000000,2.000000
max,14981.000000,100.000000,4.250334e+09,40.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [7]:
df.describe(include='object')

,name,class,id,account,twitch,ladder
count,59776,59776,59776,59776,16300,59776
unique,58286,26,59776,37123,9613,4
top,_Flic_,Berserker,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,chuz101,thespunkylobster,SSF Harbinger
freq,14,8711,1,30,18,14981


## describe result understanding (same with previous file)

##### 1. rank
rank가 이루어진 순위는 15000이나 50%에 7472로 있는 것을 볼 수 있다. 

rank의 빈 곳을 일부 채워 넣었기 때문이며, rank max 값 또한 15000이었으나 14981로 수정되어있다.

##### 2. level
이들의 평균 level 은 85이다. 최대 level이 100이지만 대부분 숙련자라는 뜻이며, 이들 중 50%에 해당하는 랭커들이 90레벨 이상이다. 또한 그들 중 25%에 해당하는 사람들이 95레벨 이상이었다. 

*(레벨은 경험이 많은 유저일 수록 레벨이 높고, 그들이 상위 랭크에 올라가있을 확률또한 높으며, 게임이 오래될 수록 신규 유저의 유입은 줄어들어서 전체 유저들의 평균 레벨 자체가 올라가기 때문에 크게 의미를 두지 않을 생각이다.)*

##### 3. exp
경험치는 레벨 자체와 비슷하게 여길 수 있으나 평균이 1.825786e+09, 50%가 1.934010e+09, max값이 4.250334e+09 이다.

그러나 표준편차가 1.094748e+09이다. 가장 작은 값은 7.790982e+07 인데 이와 비교해보기만 해도 상당히 큰 표준편차인 것을 확인할 수 있다.

최소 레벨이 53임을 함께 견주어 볼때, 일반적인 RPG가 그렇듯 레벨이 올라갈 수록 경험치를 더 많이 쌓아야 레벨을 올릴 수 있음을 예상해 볼 수 있는 부분이다. 

##### 4. challenges
도전 횟수로 생각이 된다. 해당 게임 모드를 몇 번을 수행을 했는지에 대한 것으로 생각된다. 적은 사람은 4번이지만 많은 사람은 40번이며,

이 15000등 안에 드는 사람들은 평균적으로 25회 정도 시도한 사람들이었다는 것을 확인할 수 있다.

##### 5. 'special_class', 'stat_Str', 'stat_Int', 'stat_Dex', 'stats_num'
이 부분은 임의로 내가 넣은 부분으로, 클래스를 스탯으로 나누어 쉽게 구분할 수 있도록 만들었다. 

직업 자체도 직업 자체지만, 해당 스킬을 찍기 위해 스탯을 올린다던지, 그렇게 올린 스탯을 더 잘 활용하기 위해 직업을 선택했을 수 있는 부분이 보였기 때문이다.

##### 6. name
비슷한 이름을 쓰는 경우가 많은 가보다. '_Flic_' 이라는 이름이 등장한 것은, 인기있는 이름이었거나 혹은 동일 유저가 많은 계정을 썼을 수도 있다.

확실한 건 이 게임은 같은 닉네임을 허용하는 게임인 것이라는 점이다.

##### 7. class
19개 의 전직 클래스와 7개의 전직 이전 클래스 총 26개가 보인다.

가장 많이 등장한 직업은 스탯 중 힘만 찍는 Marauder 계열의 'Berserker'이고 총 8711 번 등장한다고 한다.

##### 8. id and account
id는 unique 하다. 게임 내에서 캐릭터가 구분되어야 하기 때문일 것이다. 하지만 한 계정이 최대 30번 등장한다. 계정당 만들 수 있는 캐릭터가 제한이 없...나? (게임마다 캐릭터 슬롯을 확장하는 데에 현금을 요구하기도 하기 때문에 이해하지만 30개라니. 다양한 스탯으로 다양한 직업을 할 수 있는 점에서 수긍은 가는 부분.)

30개를 만든 사람은 바로 'chuz101'이라는 유저이다.

재미있는 부분은 6만 여개에 이르는 랭커 순위에서, 사실 unique account, 즉 실제 유저 자체는 6만 여명이 아니라 37123명인 점이다. 

그리고 이 또한 같은 사람이 여러 account를 쓸 가능성도 있기 때문에 실제 랭커는 더욱 줄어들 수 있다. (twitch 아이디로 찾아보면 나오겠지...?)

##### 9. twitch
유일하게 null 값이 있었던 컬럼이다. 실시간 스트리밍을 했는지 안했는지는 모르지만 해당 유저가 자신의 트위치 아이디를 등록을 했는지 안했는지에 따라 나오는 것으로 보인다. 

정확한 사실을 알면 더 좋겠지만, 이 컬럼 자체로 이 사람이 스밍을 하는 트위치 유저였는지 아니면 입력하래서 가지고 있는 트위치 계정을 입력했을 뿐인지는 큰 차이이기 때문에 동일 유저를 판별하는 부분에서만 사용할 수 있을 것으로 생각된다.

위에 계정 개수를 생각했을 때 유니크값이 9613개인 것을 보면 랭커 4명중 1명은 트위치 계정을 갖고있다.

그리고 총 16300 레코드가 기록되어 있으므로 같은 유저의 다 계정이 확실시 되는 부분이다.

'thespunkylobster' 라고 성급한 바닷가재라는 사람이 총 18번 등장했다고 한다.

##### 10. ladder
이전에 자세히 다루었던 부분이지만, 이 데이터를 크게 구분짓는 줄기라고 생각된다. 왜냐하면 게임모드이고, 게임모드의 난이도나 파티플레이 가능 여부는 게임 플레이에서 큰 영향을 끼친다.

숙련되지 않은 사람을 버스태울(!)수도 있고, 클래스별로 분포도 갈릴 것이다. 이후 자세히 볼 예정이다.

---

*분석의 목적을 정확히 정하고 싶지만 이 게임에서 랭킹에 드는 기준조차 제대로 모르기 때문에, 랭커들의 직업이나 특성들을 찾아보는 것을 주 목적으로 하되, 데이터로 게임(맥락)을 이해할 수 있을지 궁금한 부분도 있어서 최대한 많은 생각을 하면서 분석을 진행하려 한다.*

In [8]:
df.nunique()

rank             14981
dead                 2
online               2
name             58286
level               48
class               26
id               59776
experience       54124
account          37123
challenges          37
twitch            9613
ladder               4
special_class        2
stat_Str             2
stat_Int             2
stat_Dex             2
stats_num            3
dtype: int64

In [9]:
df.isnull().sum()

rank                 0
dead                 0
online               0
name                 0
level                0
class                0
id                   0
experience           0
account              0
challenges           0
twitch           43476
ladder               0
special_class        0
stat_Str             0
stat_Int             0
stat_Dex             0
stats_num            0
dtype: int64

# Division with 'class' in this file

## class divising conditions

```
각 클래스 나누기 (전직하지 않은 사람들도 있기 때문에 각 전직 이전 직업도 넣어준다.)

# 한 개인 직업
Marauder = ['Marauder', 'Juggernaut', 'Berserker', 'Chieftain'] # Strength
Witch = ['Witch', 'Necromancer', 'Elementalist', 'Occultist'] # Intelligence
Ranger = ['Ranger', 'Raider', 'Deadeye', 'Pathfinder'] # Dexterity

# 두 개인 직업
Duelist = ['Duelist', 'Slayer', 'Gladiator', 'Champion'] # Strength, Dexterity
Shadow = ['Shadow', 'Assassin', 'Saboteur', 'Trickster'] # Dexterity, Intelligence
Templar = ['Templar', 'Inquisitor', 'Hierophant', 'Guardian'] # Intelligence, Strength

# 세 개인 직업
Scion = ['Scion', 'Ascendant'] # Strength, Dexterity, Intelligence
```

In [10]:
"""
이후 데이터를 조금 쉽게 활용하기 위해 나누었던 컬럼을 사용할 부분을 적어둔다.
"""

# 전직을 선택하지 않은 직업
con_pre_classes = df['special_class'] == 0

# 각 스탯을 찍은 상태
con_str = df['stat_Str'] == 1
con_int = df['stat_Int'] == 1
con_dex = df['stat_Dex'] == 1

In [13]:
# condition 확인용 Marauder

df[con_str & ~con_int & ~con_dex]['class'].unique()

array(['Chieftain', 'Berserker', 'Juggernaut', 'Marauder'], dtype=object)

# each stats distribution

## Only one stats

In [28]:
# 힘에 올인한 랭커유저는 전체랭커 중 얼마나 될까?

print("Marauders(Only Strenth characters) rate :")
print(df[con_str & ~con_int & ~con_dex]['id'].count()/df['id'].count() * 100)

Marauders(Only Strenth characters) rate :
21.69097965738758


In [29]:
# 민첩에 올인한 랭커유저는 전체랭커 중 얼마나 될까?

print("Witches(Only Dexterity characters) rate :")
print(df[~con_str & ~con_int & con_dex]['id'].count()/df['id'].count() * 100)

Witches(Only Dexterity characters) rate :
16.99009635974304


In [30]:
# 지능에 올인한 랭커유저는 전체랭커 중 얼마나 될까?

print("Rangers(Only Intellegence characters) rate :")
print(df[~con_str & con_int & ~con_dex]['id'].count()/df['id'].count() * 100)

Rangers(Only Intellegence characters) rate :
16.317585653104924


## Two stats

# ladder

In [ ]:
df['ladder'].describe()

In [ ]:
df['rank'].value_counts()

In [ ]:
df['rank'].value_counts().max()

각 랭크는 최대 4명으로 되어있다. 이것만 보아도 게임 모드에 따라 랭크가 구분되어 있음을 미루어 짐작해 볼 수 있다. 확인해본다.

In [ ]:
pd.pivot_table(data = df, index = ['ladder'], aggfunc = 'count')[['account', 'rank']]

아이디의 개수와 랭크의 개수가 일치하는 것으로 나왔다. 게임 모드에 따라 랭크가 구분되어 있음이 맞다.

개인적인 생각으로, 이 데이터는 rank별로 나열될 것이 아니라, ladder 별로 rank가 나열되어야 하는 것이 맞다고 생각이 된다.



In [ ]:
# 각 모드의 id, rank 고유 개수

modes = ['Harbinger', 'Hardcore Harbinger', 'SSF Harbinger', 'SSF Harbinger HC']

for i in modes:
    print("ladder: ", i)
    print("values of unique id: ", df[df['ladder'] == i]['id'].nunique())
    print("values of unique rank: ", df[df['ladder'] == i]['rank'].nunique())
    print("max values of rank: ", df[df['ladder'] == i]['rank'].max())
    print('-------------------------------')

In [ ]:
df[df['ladder'] == 'Harbinger']['rank']

분명 모든 게임 모드에서 15000 등까지 있는 것으로 나타나지만 nunique를 넣어보면 일부 사라져 있다.

랭킹 개수 < 최대 랭크넘버 이기 때문에 동점자가 있는 것 같지는 않고, 크롤링 상의 문제이거나, 내부 데이터 상의 어떤 그런 것에 의해서 일부 없는 것으로 생각된다. (게임모드에서는 결측치가 없었기 때문)

등수는 절대적인 수치가 아니고, 나열 되어서 순서가 있을 때 의미가 존재하기 때문에 없는 등수를 채워서 총 14918 등까지 나타나도록 수정할 것이다.

## df divising

In [ ]:
df_Har = df[df['ladder'] == 'Harbinger']
df_HarHC = df[df['ladder'] == 'Hardcore Harbinger']
df_SHar = df[df['ladder'] == 'SSF Harbinger']
df_SHarHC = df[df['ladder'] == 'SSF Harbinger HC']

In [ ]:
df_Har

In [ ]:
df_Har['rank'].nunique()

rank 는 15000 까지 있지만 row는 12918 로 일부 유실되어있다. 이는 df_Har 뿐만이 아닐 것이다.

어떠한 이유에서든 없는 것은 지금 다시 가져올 수 없으므로, 빠지는 값이 없이 해주는 것이 더 정확한 계산을 가져올 수 있을 것이라 생각한다.

## Kinds of Path of Exile classes

해당 데이터셋에서, poe에는 ['Necromancer', 'Deadeye', 'Raider', 'Occultist', 'Pathfinder',
       'Guardian', 'Champion', 'Gladiator', 'Inquisitor', 'Elementalist',
       'Slayer', 'Berserker', 'Juggernaut', 'Ascendant', 'Trickster',
       'Hierophant', 'Chieftain', 'Saboteur', 'Assassin', 'Ranger',
       'Marauder', 'Scion', 'Witch', 'Duelist', 'Shadow', 'Templar']  총 24개의 unique 클래스가 존재하며,

POE 공식 사이트를 참고한 결과 각 직업별 클래스는 이렇게 나뉘고 있다.

    Duelist - 'Slayer', 'Gladiator', 'Champion'
    Shadow - 'Assassin', 'Saboteur', 'Trickster'
    Marauder(약탈자 라고 한다) - 'Juggernaut', 'Berserker', 'Chieftain',
    Witch - 'Necromancer', 'Elementalist', 'Occultist'
    Ranger - 'Raider', 'Deadeye', 'Pathfinder'
    Templar - 'Inquisitor', 'Hierophant', 'Guardian'
    Scion - 'Ascendant'
    
일반적으로 레인저는 원딜러, 듀얼리스트는 전사로 보이며, 위치는 마딜러, 템플러는 수도승 같은 등등 다른 게임에서 보이는 직업과 비슷한 부분이 있지만,

각 클래스가 가진 특성들도 다르기 때문에 poe에서 제공하고있는 클래스 별로도 나눌 수 있을 것이고, 일반적인 rpg 특성에 따라 이를 나누는 방법은 이런 방식들이 될 것이다.

    1. 탱커, 딜러, 힐러(보조직업)
    2. 탱커, 일반딜러, 암살딜러, 힐러(보조직업)
    2. 탱커, 근접딜러, 원거리딜러, 힐러(보조직업)
    3. 탱커, 근접딜러, 원거리물리딜러, 원거리마법딜러, 힐러(보조직업)
    4. 근거리, 원거리, 마법, 힐러(보조직업)
    5. 점사딜스킬위주, 광역딜스킬위주, 힐러(보조직업) ... 등등

이에 따라 각 클래스를 읽어보고 데이터 labeling을 진행하도록 한다.

스킬 범위나 세부 효과에 관련한 부분은 실제로 poe를 해본 사람이 분석하는 것이 더 바를 것 같다는 판단이 든다.

클래스의 호불호 혹은 성능을 떠나서, 세부적인 수치와 관련해서는 플레이 해보지 않는 이상 체감을 해볼 수 없기 때문이다.

예를 들어 템플러의 가디언은 방어 버프 혹은 적의 버프를 방해하는 스킬들이 있는데 상세히 알기는 어렵다.

---

https://hong-sam.tistory.com/27 글을 참조하여 일반적인 직업으로 나누는 것을 먼저 하기로 결정.

글에 따르면 
```
    Duelist : 빠른 근접 전투(힘, 민첩)
    Shadow : 민첩하고 지능적인 전투(민첩, 지능)
    Marauder : 전사(힘)
    Witch : 마법사(지능)
    Ranger : 궁수(민첩)
    Templar : 성기사(힘, 지능)
    Scion : 모든 스탯이 중간(힘, 민첩, 지능)
```

으로 나누어져있다. 사이온 같은 경우 성장 중 사이온을 구하는 퀘스트를 깨야 직업을 할 수 있다고 한다.

스킬을 찍는 데에 자유로운 것이 특징이나 자신의 원래 직업과 스탯에 따라 가는 것이 더 좋은 효율을 낼 수 있다는 점에서 다른 게임과 크게 다르지는 않다.

살펴본 결과 패시브 혹은 버프(디버프)스킬들도 많이 포함되어 있었다. 

자유도가 높지만 높아도 그 안에서 더 좋은 효율을 내는 일부 직업이나 스킬트리가 유행하여 해당 직업에는 한 클래스가 주를 이루기 마련인데, 그런 경향도 있는지 살펴보아야 겠다.

문제는 사이온인데, 사람마다 찍는 유형에 따라 근접일 수도 원거리일 수도, 완전히 보조 일수도, 그 중간 어딘가일 수도 있다.

여러 능력의 조합을 사용하는 것의 이점과 단점은 극명하기 때문에 숙련된 사람이 아니면 사이온을 달고 랭킹에 진입하기는 쉽지 않았을 것이라는 생각도 든다.


방식 1: 세부 클래스 전 직업(전직 전)으로 묶은 다음 그에 따라 labeling, 사이온은 사이온으로.
```
    근 - Duelist - 'Slayer', 'Gladiator', 'Champion'
    근 - Shadow - 'Assassin', 'Saboteur', 'Trickster'
    근 - Marauder - 'Juggernaut', 'Berserker', 'Chieftain',
    원 - Witch - 'Necromancer', 'Elementalist', 'Occultist'
    원 - Ranger - 'Raider', 'Deadeye', 'Pathfinder'
    원 - Templar - 'Inquisitor', 'Hierophant', 'Guardian'
    사이온 - Scion - 'Ascendant'
```
방식 2: 세부 클래스(전직 후)에 따라 labeling (딜 위주, 탱킹 위주, 보조 위주 - 게임 공부를 더 하고 나누는 걸로. EDA 하다보면 아이디어를 얻을수도 있고,)

    'Slayer' : 양손 무기, 생존력 특화 Duelist
    'Gladiator' : 쌍수 혹은 한손-방패 Duelist
    'Champion' : 적 디버프, 자신 버프 Duelist
    'Assassin' : 출혈, 독 등 이용한 치명 딜 위주 Shadow
    'Saboteur' : 트랩, 마인 활용 Shadow
    'Trickster' : 기동성, 회피, 공격속도 위주 Shadow
    'Juggernaut' : 탱킹 중심 Marauder
    'Berserker' : 흡혈이나 피해 비례 데미지 등 광전사 Marauder
    'Chieftain' : 화염, 적 디버프 이용 Marauder
    'Necromancer' : 소환술 특화 Witch
    'Elementalist' : 원소 피해 특화 Witch
    'Occultist' : 카오스 피해, 저주 특화 Witch
    'Raider' : 회피, 처치 후 얻는 버프를 이용하는 Ranger
    'Deadeye' : 치명타, 딜 중심 Ranger
    'Pathfinder' : 플라스크(물약), 치유효과 강화, 디버프 제거, 원소, 카오스 이용 Ranger
    'Inquisitor' : 원소 치명타, 마나 재생, 자신 버프 Templer
    'Hierophant' : 토템 및 마나 변환, 받은 피해 마나로 감소 등의 Templer
    'Guardian' : 서포트 및 보호(방어버프, 디버프제거, 치유 등) 특화 Templer
    'Ascendant' : 타 클래스들의 조합하는 Scion. 성능은 다소 떨어질 수 있음.

방식 3: 특정 스탯 한 가지 / 혹은 두 가지 / 세 가지

    'Marauder', 'Witch', 'Ranger'
    'Duelist', 'Shadow', 'Templar'
    'Scion'

In [ ]:
df_byladder['class'].unique()

전직 전과 전직 후 *모두 섞여있는 것* 또한 볼 수 있다. 이에 따라 이를 기준으로 나눌 수도 있다. 
전직 전과 후로 찍을 수 있는 스킬이 다를 것이기 때문이다. (물론 젬이 필요하다고 했기 때문에 그것이 없어서 그 직업을 운용중일 지도 모른다)

# class labeling

## ['special_class']: before and after to select special classes

In [ ]:
df

In [ ]:
df['special_class'] = 0

In [ ]:
pre_classes = ['Duelist', 'Shadow', 'Marauder', 'Witch', 'Ranger', 'Templar', 'Scion']

In [ ]:
df['special_class'] = np.where(df['class'].isin(pre_classes), 0, 1)

In [ ]:
df.head(10)

### How many ranked users in there who didn't select the special class?

In [ ]:
df['special_class'].value_counts()

In [ ]:
# 위와 같은 결과를 그룹바이로 낼 수 있다.

freq = df.groupby(['special_class']).size() 
print(freq)

### Then, what pre-classes of them are there?

In [ ]:
df[df['special_class'] == 0]['class'].value_counts()

위 결과를 보니 더더욱! 라벨링을 하는게 좋겠다는 생각이 드는데, 게임을 어떤 관점에서 볼지에 따라 나누는 방식을 달리할 수 있을 것이다.

포지션 자체인지 아니면 전체적인 스킬의 유형(사거리 등)에 관한 것인지 사람마다 중요시 하는 부분이 다르기 때문에... 

살펴보면 살펴볼 수록 이 게임은 사거리 자체보다는 포지션이나, 솔플 난이도 여부에 따라 갈리는게 나을 것 같다는 생각이 든다. 그래서 SSF 모드가 따로 있는 것이기도 한 이유일 것이다.

### What play modes of them are there?

In [ ]:
df[df['special_class'] == 0]['ladder'].value_counts()

독특하게도, 전직하지 않은 사람들은 일반 Harbinger 에는 없었고, 대부분 솔로 플레이모드에 많았다.

아무래도 전직이라는 건 세부 직업을 고르는 것, 즉 특화를 하는 것이기 때문에 특화되지 않은 다른 면에서는 부족해질 수 있다고 생각해보면, 

솔로플레이에서는 전직하지 않은 직업으로 플레이 하는게 더 나을 수도 있다는 생각이다. 물론 스킬을 찍는 포인트가 한정되어있다는 사실 안에 말이다.



###  df_pre_cl : stats description by whether pre or post special classes

전직을 하지 않은 사람들과 한 사람들의 통계적 요약을 보고자 한다.

In [ ]:
df_pre_cl = df[df['special_class'] == 0]
df_pre_cl

In [ ]:
df_pre_cl.describe()

평균 랭크 자체는 높은 편이아니고, 레벨의 최대 값은 94였다. 최대 레벨이 100인 점을 감안할 때 물음표가 생길 수 밖에 없는 부분이다. 게임을 정확히 이해하지 않고서는 풀기 어려운 의문인 것 같다.

반대로 전직을 한 사람들을 보자면, 

In [ ]:
df[df['special_class'] == 1].describe()

세부 클래스로 전직한 평균 랭크가 약 3천 이상 더 높고, 레벨도 평균적으로 더 높다. 

하지만 경험치는 그들보다 평균적으로 더 적었고, 시도도 더 많이 한 것으로 나타났다.

이를 토대로 합리적의심을 해보자면, 숙련도나 게임 경력에 따라서 전직 여부를 결정하지는 않았을 것이라고 추측해 볼 수 있다.

## ['stat_Str', 'stat_Int', 'stat_Dex']: columns by kind of stats and stats numbers

먼저, 특정 스탯 한 가지 / 혹은 두 가지 / 세 가지

    'Marauder', 'Witch', 'Ranger'
    'Duelist', 'Shadow', 'Templar' 
    'Scion'

에 따라 stat_num(개수) 을 1, 2, 3으로 나누게 될 것이다.

각 클래스에 따라 어떤 특성을 찍었는지 특성(힘, 민첩, 지능) 을 나누어서 여부를 라벨링 한 다음 sum 으로 낼 계획


In [ ]:
# 각 클래스 나누기 (전직하지 않은 사람들도 있기 때문에 각 전직 이전 직업도 넣어준다.)

# 한 개인 직업
Marauder = ['Marauder', 'Juggernaut', 'Berserker', 'Chieftain'] # Strength
Witch = ['Witch', 'Necromancer', 'Elementalist', 'Occultist'] # Intelligence
Ranger = ['Ranger', 'Raider', 'Deadeye', 'Pathfinder'] # Dexterity

# 두 개인 직업
Duelist = ['Duelist', 'Slayer', 'Gladiator', 'Champion'] # Strength, Dexterity
Shadow = ['Shadow', 'Assassin', 'Saboteur', 'Trickster'] # Dexterity, Intelligence
Templar = ['Templar', 'Inquisitor', 'Hierophant', 'Guardian'] # Intelligence, Strength

# 세 개인 직업
Scion = ['Scion', 'Ascendant'] # Strength, Dexterity, Intelligence